# Q-Learning with Taxi-v3 🚕


In this notebook, you'll implement a Q-Learning taxi that will need to learn to navigate in a city to transport its passengers from point A to point B.


## Import the libraries 📚


In [39]:
import numpy as np
import random
import gym
import matplotlib.pyplot as plt

## Create the environment 

In [36]:
env = gym.make('Taxi-v3')
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Create and initialize the Q-Table

In [4]:
# all state space = 6
# 4 directions (North, South, East, West) and Pickup and Put down
action_space = env.action_space.n 
print("There are",action_space,"possible states")


observation_space = env.observation_space.n
print("There are",observation_space,"possible states")

There are 6 possible states
There are 500 possible states


In [5]:
#Initialize my q-table to 0
qtable = np.zeros((observation_space,action_space))
print(qtable)
print(qtable.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
(500, 6)


## Define the hyperparameters

In [6]:
total_episodes = 10         # Total number of training episodes
total_test_episodes = 10     # Total number of test episodes
max_steps = 10               # Max steps per episode

learning_rate = 0.01          # Learning rate
gamma = 0.99                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.001           # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

## Function greddy policy 

The idea is that we define epsilon ɛ = 1.0:
- With probability 1 — ɛ : we do exploitation (our agent selects the action with the highest state-action pair value).
- With probability ɛ: we do exploration (trying random action).

In [7]:
def greddy_policy(Q,state):
    #Exploitation
    if(random.uniform(0,1) > epsilon):
        action = np.argmax(Q[state])
    
    #Because epsilon is big, I take a random action. 
    #Exploration
    else:
         action = env.action_space.sample()
    return action

## The Q-Learning algorithm 

In [26]:
for episode in range(total_episodes): #episode 
    step = 0
    final = False
    #Reset environnement
    state = env.reset()
    
    #Reduce epsilon
    epsilon = min_epsilon + (max_epsilon - min_epsilon)* np.exp(-decay_rate * episode)
    
    for step in range(max_steps): # step 
        
        action = greddy_policy(qtable,state)
        
        # Take action (a) and get reward (r), transit to next state (s)

        new_state, reward , done , info = env.step(action)
        
        # TD update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        qtable[state][action] = qtable[state][action] + learning_rate * (reward + gamma *
                                                                         np.max(qtable[new_state]-qtable[state][action]))
        
        if final == True:
            break
            
        state = new_state 

## Evalute the agent 